<a href="https://colab.research.google.com/github/gladcolor/lowest_floor_elevation/blob/gladcolor-patch-1/lowest_floor_elevation_vertical_IoU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
% cd /content/drive/MyDrive/Research/Lowest_first_floor
% pwd

/content/drive/MyDrive/Research/Lowest_first_floor


'/content/drive/MyDrive/Research/Lowest_first_floor'

In [ ]:
import os
import glob
import pandas as pd

origin_dir = r'origin/'
detected_dir = r'detected/'

origin_files = glob.glob(os.path.join(origin_dir, "*.txt"))
detected_files = glob.glob(os.path.join(detected_dir, "*.txt"))


def iou(bbox1, bbox2, vertical=True, horizontal=True):  # bbox: (top, right, bottom, left)
    top1, right1, bottom1, left1 = bbox1
    top2, right2, bottom2, left2 = bbox2

def is_intersect(bbox1, bbox2): # bbox: (top, right, bottom, left)
  top1, right1, bottom1, left1 = bbox1
  top2, right2, bottom2, left2 = bbox2
  return ~((top2 <= bottom1) or (top1 <= bottom2) or (left1 >= right2) or (left2 >= right1))

def find_intersect_bbox(bbox, bboxes):
  intersected = []
  for b in bboxes:
    if is_intersect(bbox, b):
      intersected.append(b)
  return intersected

def box_area(box):
  return (box[2] - box[0]) * (box[3] - box[1])

def box_iou(box1, box2):  # d_bbox, o_bbox
  # print(box1, box2)

  iou_dict = {}

  i_top = min(box1[0], box2[0])
  i_right = min(box1[1], box2[1])
  i_bottom = max(box1[2], box2[2])
  i_left = max(box1[3], box2[3])
  # print("i_top, i_right, i_bottom, i_left:", i_top, i_right, i_bottom, i_left)
  # compute the area of intersection rectangle
  inter = (i_right - i_left) * (i_top - i_bottom)

  area1 = box_area(box1)
  area2 = box_area(box2)
  iou_area = inter / (area1 + area2 - inter)

  iou_dict["iou_area"] = iou_area

  # iou vertical:
  u_top = max(box1[0], box2[0]) 
  u_bottom = max(box1[2], box2[2]) 
  iou_vertical = (i_top - i_bottom) / (u_top - u_bottom)

    # bottom_error:
  bottom_diff = abs(box1[2] - box2[2])   
  bottom_error = bottom_diff / (box1[0] - box1[2])

  # top_error:
  top_diff = abs(box1[0] - box2[0])   
  top_error = top_diff / (box1[0] - box1[2])

  iou_dict["iou_area"] = iou_area
  iou_dict["iou_vertical"] = iou_vertical
  iou_dict["bottom_error"] = bottom_error
  iou_dict["top_error"] = top_error

  # if (iou ):
  #   print("d_bbox, o_bbox:", box1, box2)
  #   print("IoU:", iou)
  #   print("i_top, i_right, i_bottom, i_left:", i_top, i_right, i_bottom, i_left)
  #   print("")

  return iou_dict

def find_intersected_pairs(detected_df, origin_df):
  categories = detected_df['category'].unique()
  pairs = []
  for c in categories:
    d_df = detected_df[detected_df['category'] == c]
    o_df = origin_df[origin_df['category'] == c]

    o_bboxes = []
    for idx, row in  o_df.iterrows():
      o_bboxes.append((row["top"], row["right"], row["bottom"], row["left"]))

    for idx, row in  d_df.iterrows():
      d_bbox = (row["top"], row["right"], row["bottom"], row["left"])

      for o_bbox in o_bboxes:
        if is_intersect(d_bbox, o_bbox):
          pairs.append((c, d_bbox, o_bbox))

  return pairs

def read_origin_bbox():

  return 0

def form_bbox_from_df(df):
  try:
    df["top"]    = df["center_y"] + df["height"] / 2
    df["bottom"] = df["center_y"] - df["height"] / 2

    df["right"]  = df["center_x"] + df["width"] / 2    
    df["left"]   = df["center_x"] - df["width"] / 2



  except Exception as e:
    print("Error in form_bbox_from_df():", e)
  return df

def calculate_iou_from_detected(origin_dir, detected_dir):
  origin_files = glob.glob(os.path.join(origin_dir, "*.txt"))
  # detected_files = glob.glob(os.path.join(detected_dir, "*.txt"))

  # get detected_files
  test_files = open(r'test.txt', 'r').readlines()
  test_files = [f[:-5] + ".txt" for f in test_files]
  test_files = [os.path.join(detected_dir, os.path.basename(f)) for f in test_files]

  print(test_files)

  detected_files = test_files
  # print(test_files)
  # test_files = [os.path.join(detected_dir, os.path.basename(f) for f in test_files]
  # test_files = [test_files[:-5] + ".txt"]
  # print(test_files)

  # detected_files = [os.path.join(detected_dir, "31795.txt")]
  iou_list = []
  for idx, d_file in enumerate(detected_files):
    o_file = os.path.join(origin_dir, os.path.basename(d_file))
    if not os.path.exists(o_file):
      continue
    if not os.path.exists(d_file):
      continue
    # read origin bbox
    origin_df = pd.read_csv(o_file, header=None, sep=" ")
    origin_df.columns = ["category", "center_x", "center_y", "width", "height"]
    origin_df = form_bbox_from_df(origin_df)

    # print(origin_df)

    d_lines = open(d_file, 'r').readlines()
    o_lines = open(o_file, 'r').readlines()

    # print(d_lines)


    detected_df = pd.read_csv(d_file, header=None, sep=" ")
    detected_df = detected_df[range(6)]
    # print(detected_df)
    detected_df.columns = ["category", "confidence", "center_x", "center_y", "width", "height"]
    detected_df = form_bbox_from_df(detected_df)

    # print(detected_df)

    target_category_idx = 0 # door

    

    intersected_pairs = find_intersected_pairs(detected_df[detected_df["category"].isin([target_category_idx])], origin_df[origin_df["category"].isin([target_category_idx])])


    
    for p in intersected_pairs:
      iou_dict = box_iou(p[1], p[2])
      # print(iou)
      iou_list.append(iou_dict)
      # if iou :
      #   print(detected_df[detected_df["category"] == 0])
      #   print(os.path.basename(d_file))

      #   print("")
      # print(p[0])
      # print(p)
      # print("IoU:", box_iou(p[1], p[2]))
  return iou_list
      
    


iou_list = calculate_iou_from_detected(origin_dir, detected_dir)



['detected/2281.txt', 'detected/2290.txt', 'detected/2296.txt', 'detected/13788.txt', 'detected/13795.txt', 'detected/13803.txt', 'detected/13808.txt', 'detected/21615.txt', 'detected/21626.txt', 'detected/22476.txt', 'detected/22486.txt', 'detected/22491.txt', 'detected/29095.txt', 'detected/29100.txt', 'detected/29105.txt', 'detected/31074.txt', 'detected/31079.txt', 'detected/31086.txt', 'detected/31092.txt', 'detected/31098.txt', 'detected/31103.txt', 'detected/31109.txt', 'detected/31120.txt', 'detected/31130.txt', 'detected/31135.txt', 'detected/31140.txt', 'detected/31146.txt', 'detected/31152.txt', 'detected/31159.txt', 'detected/31165.txt', 'detected/31171.txt', 'detected/31177.txt', 'detected/31182.txt', 'detected/31187.txt', 'detected/31196.txt', 'detected/31201.txt', 'detected/31206.txt', 'detected/31211.txt', 'detected/31216.txt', 'detected/31221.txt', 'detected/31227.txt', 'detected/31232.txt', 'detected/31238.txt', 'detected/31244.txt', 'detected/31253.txt', 'detected/31

In [ ]:
import numpy as np

iou_dict_all = {'bottom_error': 0, "top_error": 0,
  'iou_area': 0,
  'iou_vertical': 0,
  "top_error_list": [],
  "bottom_error_list": []}

for iou in iou_list:
  iou_dict_all['bottom_error'] += iou['bottom_error']
  iou_dict_all['iou_area'] += iou['iou_area']
  iou_dict_all['iou_vertical'] += iou['iou_vertical']
  iou_dict_all['top_error'] += iou['top_error']
  iou_dict_all['top_error_list'].append(iou['top_error'])
  iou_dict_all['bottom_error_list'].append(iou['bottom_error'])

iou_dict_all['bottom_error'] = iou_dict_all['bottom_error'] / len(iou_list)
iou_dict_all['iou_area'] = iou_dict_all['iou_area'] / len(iou_list) 
iou_dict_all['iou_vertical'] = iou_dict_all['iou_vertical'] / len(iou_list) 
iou_dict_all['top_error'] = iou_dict_all['top_error'] / len(iou_list) 



iou_dict_all['bottom_std'] = np.array(iou_dict_all['bottom_error_list']).std() 
iou_dict_all['top_std'] = np.array(iou_dict_all['top_error_list']).std() 

print("iou_dict_all['top_std']:", iou_dict_all['top_std'])
print("iou_dict_all['top_error']:", iou_dict_all['top_error'])

print("iou_dict_all['bottom_std']:", iou_dict_all['bottom_std'])
print("iou_dict_all['bottom_error']:", iou_dict_all['bottom_error'])

print("iou_dict_all['iou_area']:", iou_dict_all['iou_area'])
print("iou_dict_all['iou_vertical']:", iou_dict_all['iou_vertical'])



iou_dict_all['top_std']: 0.06931612709490012
iou_dict_all['top_error']: 0.04689380791835483
iou_dict_all['bottom_std']: 0.0474653518042457
iou_dict_all['bottom_error']: 0.027458709639779522
iou_dict_all['iou_area']: 0.8146327722249954
iou_dict_all['iou_vertical']: 0.9508862843846395


In [ ]:
origin_dir = r'origin/'
detected_dir = r'detected/'

def calculate_precision(origin_dir, detected_dir):
  origin_files = glob.glob(os.path.join(origin_dir, "*.txt"))
  # detected_files = glob.glob(os.path.join(detected_dir, "*.txt"))

  # get detected_files
  test_files = open(r'test.txt', 'r').readlines()
  test_files = [f[:-5] + ".txt" for f in test_files]

  test_files = [os.path.join(detected_dir, os.path.basename(f)) for f in test_files]
  origin_files = [os.path.join(origin_dir, os.path.basename(f)) for f in test_files]

  print(origin_files)

  detected_files = test_files

  origin_polygon_cnt = 0
  detected_polyugon_cnt = 0

  metric = {"origin_polygon_cnt": 0, "detected_polygon_cnt": 0, "intersected_bbox_cnt": 0, 
            "recall": 0, 
            "precision": 0
            }

  target_category_idxs = [0] # door

  for idx, o_file in enumerate(origin_files):
    d_file = os.path.join(detected_dir, os.path.basename(o_file))

    # read origin bbox
    origin_df = pd.read_csv(o_file, header=None, sep=" ")
    origin_df.columns = ["category", "center_x", "center_y", "width", "height"]

    origin_df = origin_df[origin_df["category"].isin(target_category_idxs)]

    origin_df = form_bbox_from_df(origin_df)


    metric["origin_polygon_cnt"] += len(origin_df)


    if not os.path.exists(d_file):
      continue

    d_lines = open(d_file, 'r').readlines()
    o_lines = open(o_file, 'r').readlines()

    detected_df = pd.read_csv(d_file, header=None, sep=" ")
    # print(detected_df)
    detected_df = detected_df[range(6)]
    # print(detected_df)
    detected_df.columns = ["category", "confidence", "center_x", "center_y", "width", "height"]
    detected_df = detected_df[detected_df["category"].isin(target_category_idxs)]


    detected_df = form_bbox_from_df(detected_df)


    metric["detected_polygon_cnt"] += len(detected_df)

    intersected_pairs = find_intersected_pairs(detected_df, origin_df)

    metric["intersected_bbox_cnt"] += len(intersected_pairs)

    # if len(origin_df) != len(intersected_pairs):
      # print(d_file)
      # print("len(origin_df), len(intersected_pairs):", len(origin_df), len(intersected_pairs))
      # print("")


  metric["recall"] = metric["intersected_bbox_cnt"] / metric["origin_polygon_cnt"]
  metric["precision"] = metric["intersected_bbox_cnt"] / metric["detected_polygon_cnt"]
  return metric

calculate_precision(origin_dir, detected_dir)

['origin/2281.txt', 'origin/2290.txt', 'origin/2296.txt', 'origin/13788.txt', 'origin/13795.txt', 'origin/13803.txt', 'origin/13808.txt', 'origin/21615.txt', 'origin/21626.txt', 'origin/22476.txt', 'origin/22486.txt', 'origin/22491.txt', 'origin/29095.txt', 'origin/29100.txt', 'origin/29105.txt', 'origin/31074.txt', 'origin/31079.txt', 'origin/31086.txt', 'origin/31092.txt', 'origin/31098.txt', 'origin/31103.txt', 'origin/31109.txt', 'origin/31120.txt', 'origin/31130.txt', 'origin/31135.txt', 'origin/31140.txt', 'origin/31146.txt', 'origin/31152.txt', 'origin/31159.txt', 'origin/31165.txt', 'origin/31171.txt', 'origin/31177.txt', 'origin/31182.txt', 'origin/31187.txt', 'origin/31196.txt', 'origin/31201.txt', 'origin/31206.txt', 'origin/31211.txt', 'origin/31216.txt', 'origin/31221.txt', 'origin/31227.txt', 'origin/31232.txt', 'origin/31238.txt', 'origin/31244.txt', 'origin/31253.txt', 'origin/31259.txt', 'origin/31265.txt', 'origin/31271.txt', 'origin/31276.txt', 'origin/31284.txt', 'o

{'detected_polygon_cnt': 190,
 'intersected_bbox_cnt': 152,
 'origin_polygon_cnt': 254,
 'precision': 0.8,
 'recall': 0.5984251968503937}